In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('test.csv')

# df_filled = df.fillna(0)

# Remove the first column
# df_filled = df_filled.drop(df_filled.columns[0], axis=1)

# Save the cleaned dataset to a new CSV file
# df_filled.to_csv('training_dataset.csv', index=False)
df = df.drop(columns=['Name','Cabin'])
df['Sex'] = df['Sex'].astype('category').cat.codes
# df.drop_duplicates(inplace = True)
# print(df.to_string()) 
# print(df.isnull().sum()) 
print("------------------------------------------------")
# df = df.dropna(subset=['Fare'])
df['Embarked'] = df['Embarked'].astype('category').cat.codes
# df.replace('', pd.NA, inplace=True) 
# print(df.isna().sum())
df['Ticket'] = df['Ticket'].str.extract(r'(\d+)(?!.*\d)')
# print(df.to_string()) 
print("------------------------------------------------")
removed_data = df[df.isna().any(axis=1)]  # Select rows with any NaN
cleaned_data = df.dropna()  # Remove rows with NaN
# print("Total rows after:", len(cleaned_data))
cleaned_data.to_csv('processed_test_data.csv', index=False)
removed_data.to_csv('missing_test.csv', index=False)
# # print(cleaned_data.to_string())

------------------------------------------------
------------------------------------------------


In [4]:
# print(cleaned_data)

age_x=[]
fare_x=[]
ticket_x=[]
y=[]
age = []
fare = []
ticket=[]
with open("processed_train_data.csv", "r") as file:
    content = csv.reader(file)
    next(content)

    for line in content:
        y.append(int(line[0]))#PassengerId
        x0 = int(line[1])     #survived
        x1 = int(line[2])     #Pclass
        x2 = int(line[3])     #Sex
        x3 = float(line[4])   #Age ---
        x4 = int(line[5])     #Sibsp
        x5 = int(line[6])     #Parch  
        x6 = int(line[7])   #Ticket ---
        x7 = float(line[8])   #Fare ---
        x8 = int(line[9])     #Embarked
        # print(x1,x2,x3,x4,x5,x6,x7,x8)
        age.append(x3)
        fare.append(x7)
        ticket.append(x6)
        age_x.append(np.array([x1,x2,x4,x5,x6,x7,x8]))
        fare_x.append(np.array([x1,x2,x3,x4,x5,x6,x8]))
        ticket_x.append(np.array([x1,x2,x3,x4,x5,x7,x8]))

def normalize (x):
    sum1 = 0
    sum2 = 0
    for i in range(len(x)):
        sum1 += x[i]
        sum2 += x[i]**2

    mean = sum1/(len(x))
    variance = sum2/(len(x)) - mean**2
    std = variance**(0.5)

    for i in range(len(x)):
        x[i] = (x[i] - mean)/std
    return mean, std
    
def Normalize (x,n=0):
    sum1 = 0
    sum2 = 0
    for i in range(len(x)):
        sum1 += x[i][n]
        sum2 += x[i][n]**2

    mean = sum1/(len(x))
    variance = sum2/(len(x)) - mean**2
    std = variance**(0.5)

    for i in range(len(x)):
        x[i][n] = (x[i][n] - mean)/std
    return mean, std
mean = [0,0,0] # age, ticket, fare
std =  [0,0,0] # age, ticket, fare

mean[0], std[0] = normalize(age)
Normalize(age_x, 4) #ticket
Normalize(age_x, 5) #fare

mean[1], std[1] = normalize(ticket)
Normalize(ticket_x, 2)#age
Normalize(ticket_x, 5)#fare

mean[2], std[2] = normalize(fare)
Normalize(fare_x, 2) #age
Normalize(fare_x, 5) #ticket

# print(age_x)
# print("------------------------------------------------------")
# print(ticket_x)
# print("------------------------------------------------------")
# print(fare_x)

(np.float64(298403.92480359145), np.float64(655772.0597341646))

In [5]:
miss_age_y=[]
miss_age_x=[]

miss_fare_y=[]
miss_fare_x=[]

miss_ticket_y= []
miss_ticket_x= []

with open("missing_test.csv", "r") as file:
    content = csv.reader(file)
    next(content)

    for line in content:
        miss_y = int(line[0]) #PassengerId
        try:
            # x0 = int(line[1])     #survived
            x1 = int(line[1])     #Pclass
            x2 = int(line[2])     #Sex
            x3 = (line[3])   #Age ---
            x4 = int(line[4])     #Sibsp
            x5 = int(line[5])     #Parch  
            x6 = (line[6])   #Ticket ---
            x7 = (line[7])   #Fare ---
            x8 = int(line[8])     #Embarked
        except ValueError:
            print(line)
        # print(x0,",",x1,",",x2,",",x3,",",x4,",",x5,",",x6,",",x7,",",x8)
        # print(x3,pd.isna(x3))
        if x3 == '':
            x7 = float(x7)
            x6 = int(x6)
            miss_age_y.append(miss_y)
            miss_age_x.append(np.array([x1,x2,x4,x5,x6,x7,x8]))
            
        if x6 == '':
            x3 = float(x3)
            x7 = float(x7)
            miss_ticket_y.append(miss_y)
            miss_ticket_x.append(np.array([x1,x2,x3,x4,x5,x7,x8]))
            
        if x7 == '':
            x3 = float(x3)
            x6 = int(x6)
            miss_fare_y.append(miss_y)
            miss_fare_x.append(np.array([x1,x2,x3,x4,x5,x6,x8]))
            
for jojo in miss_age_x:
    jojo[4] = (jojo[4]- mean[1])/std[1] #ticket
    jojo[5] = (jojo[5]- mean[2])/std[2] #fare

for jojo in miss_ticket_x:
    jojo[2] = (jojo[2]- mean[0])/std[0] #age
    jojo[5] = (jojo[5]- mean[2])/std[2] #fare
    
for jojo in miss_fare_x:
    jojo[2] = (jojo[2]- mean[0])/std[0] #age
    jojo[5] = (jojo[5]- mean[1])/std[1] #ticket

# print(miss_age_x)
# print("----------------------------------------------------------")
# print(miss_ticket_x)
# print("----------------------------------------------------------")
# print(miss_fare_x)

In [6]:
age_nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(age_x)
age_distances, age_indices = age_nbrs.kneighbors(miss_age_x)
# print(age_distances)
age_factor=[]
for distances in age_distances:
    factor=[]
    for distance in distances:
        factor.append(1/(1+distance))
    age_factor.append(factor)
miss_age = np.zeros(len(miss_age_x))
i=0
# print(age_indices.to_string())
for indices in age_indices:
    avg=0
    den=0
    j=0
    for index in indices:
#         print(index)
        avg+= age[index]*age_factor[i][j]
        den+= age_factor[i][j]
        j+=1
    avg /= den
    # print(avg)
    miss_age[i] = avg
    i += 1


# print(miss_age)
# print(age_factor)

In [7]:
fare_nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(fare_x)
fare_distances, fare_indices = age_nbrs.kneighbors(miss_fare_x)
fare_factor=[]
for distances in fare_distances:
    factor=[]
    for distance in distances:
        factor.append(1/(1+distance))
    fare_factor.append(factor)
miss_fare = np.zeros(len(miss_fare_x))
i=0
for indices in fare_indices:
    avg=0
    den=0
    j=0
    for index in indices:
        avg+= fare[index]*fare_factor[i][j]
        den+= fare_factor[i][j]
        j+=1
    avg /= den
    miss_fare[i] = avg
    i += 1
# print(miss_ticket)
# print(ticket_factor)
# print(miss_ticket_x)

In [8]:
data = [['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']]

with open("processed_test_data.csv", "r") as file:
    content = csv.reader(file)
    
    next(content)
    for line in content:
        yl = int(line[0])#PassengerId
        # x0 = int(line[1])     #survived
        x1 = int(line[1])     #Pclass
        x2 = int(line[2])     #Sex
        x3 = float(line[3])   #Age ---
        x4 = int(line[4])     #Sibsp
        x5 = int(line[5])     #Parch  
        x6 = int(line[6])   #Ticket ---
        x7 = float(line[7])   #Fare ---
        x8 = int(line[8])     #Embarked
        data.append([yl,x1,x2,x3,x4,x5,x6,x7,x8])
# print(data)
l = len(data)
for pid, x, age in zip(miss_age_y, miss_age_x, miss_age):
    a = x[4]*std[1]+ mean[1] #ticket
    b = x[5]*std[2]+ mean[2] #fare
    c = age*std[0]+ mean[0] #age
    data.append([pid, int(x[0]), int(x[1]),float(c),int(x[2]), int(x[3]), int(a), float(b), int(x[6])])

for pid, x, fare in zip(miss_fare_y, miss_fare_x, miss_fare):
    a = x[2]*std[0]+ mean[0] #age
    b = x[5]*std[1]+ mean[1] #ticket
    c = fare*std[2]+ mean[2]#fare
    data.append([pid, int(x[0]), int(x[1]),float(a),int(x[3]), int(x[4]),  int(b), float(c), int(x[6])])
# for i in range(10):
#     print(data[i+l])

In [9]:
with open("processed_test_data.csv","w") as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [10]:
print(len(data))

419
